# Balanced Random Forest Classifier

In [2]:
# Initial imports.

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import matplotlib.pyplot 
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
# Loading data

file_path = Path("C:/Users/emili/Class/Patient_Survival_Prediction/database_erd.csv")
df_psp = pd.read_csv(file_path)
df_psp.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\emili\\Class\\Patient_Survival_Prediction\\database_erd.csv'

In [ ]:
# Create our features

X = df_psp.drop('hospital_death', axis=1)


X.head()


In [ ]:
X.describe()

In [ ]:
# Create our target

y = df_psp['hospital_death']
y.value_counts()

## Split Data into Training and Testing

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [ ]:
# Check balances
print(Counter(y_train))
print(Counter(y_test))

### Balanced Random Forest Classifier with All Columns

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

# Instantiate
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fit
brfc.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)


# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual Survival", "Actual Death"], columns=["Predicted Survival", "Predicted Death"])

cm_df

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# List the features sorted in descending order by feature importance
features_rank = sorted(zip(brfc.feature_importances_, X.columns), reverse=True)
for feature in features_rank:
    print(f"{feature[1]}: ({feature[0]})")

In [ ]:
#Show the importance of each feature in a graph

Importance = pd.DataFrame(np.zeros((X_train.shape[1], 1)), columns=["Importance"], index=X_train.columns)

Importance.iloc[:,0] = brf_model.feature_importance_

Importance = importance.sort_values(by="Importance", ascending=False)[:40]

plt.figure(figsize=(15, 10))
sns.barplot(x="Importance", y=importance.index, data=importance)
plt.title("Features by Importance")
plt.show()

### Easy Ensemble Classifier with All Columns

In [ ]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
# Instantiate
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)

# Fit
eec.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
# Calculating the confusion matrix

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual Survival", "Actual Death"], columns=["Predicted Survival", "Predicted Death"])
cm_df

In [ ]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Report the accuracy scores of both models


Accuracy=[(RandomForest_accuracy*100),(EasyEnsemble_accuracy*100)]

  
df = {'Algorithm': ['Random Forest Classifier','Easy Ensemble Classifier'], \
      'Acuuracy_Score':Accuracy}


Accuracy_Scores = pd.DataFrame(data=df, columns=['Algorithm','Acuuracy_Score'], index=None)

Accuracy_Scores